### Explorative Data Analysis of Raw Dataframe
After creation of the raw dataframe with the data_gathering.py script, a look into the given data is necessary. The focus will be missing and/or implausible values, correlations and trends. Technical details on the data will be shortly discussed, as well as decisions on imputation and generation of new features for preprocessing.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress=True,formatter={'float_kind':'{:0.2f}'.format})

df_raw = pd.read_csv('../Dataset/preprocessed_data/df_final_raw_2015_2023.csv',index_col=0,parse_dates=['date'])
df_raw.info()
display(df_raw.head())
df_raw.describe()

> ### Further analysis for different data sources
>
> The weather data columns are having several issues, like missing and implausible values. From the data description of [DWD](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/solar/DESCRIPTION_obsgermany_climate_daily_solar_en.pdf) the value > "-999" indicates missing values. The value of "0 J/cm²" in one column (05792_FD_STRAHL) is deemed to be an outlier and also removed, since it is physical unlikely, and also an obvious data gap is to be seen.
> The data from Marktstammdatenregister and Tennet have no missing values and are seemingly plausible.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_raw.plot(x='date',y='03668_FD_STRAHL',figsize=(10,5),label='daily sum of diffuse solar radiation',title='Example of Given Values in Weather Station Column',ax=ax) #example of stations-file for -999 values
plt.ylabel('J /cm²',fontsize=14)
plt.xlabel('Date',fontsize=14)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_raw.loc[:,'05792_FD_STRAHL'].plot(kind='hist',figsize=(10,5),bins=100,
                                     label='FD_Strahl Column of "Zugspitze"',title='Example of Distribution in Weather Station Column')
plt.xlabel('Date',fontsize=14)
ax.legend()
plt.show()
# values to be imputed: -999, 0 , since no solar radiation at all seems physically not plausible

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_raw.plot(x='date',y='Act_in_MW',figsize=(10,5),ax=ax,title='Increase of Produced Solar Power (particulary in Summer)') 
ax.get_legend().remove()
plt.ylabel('Daily Produced Energy in MW')
plt.xlabel('Date')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_raw.plot(x='date',y='Bruttoleistung',figsize=(10,5),ax=ax,title='Increasing of Registered Solar Panel Capacity',label='Daily registered capacity') 
plt.ylabel('Capacity in kWp')
plt.xlabel('Date')
plt.show()

### Preprocessing of raw data

For further usage in this project the missing values from the weather stations needs to be imputed, and new feature are needed to be engineered.

First the values will be set to pd.NA

In [ ]:
df = pd.read_csv('../Dataset/preprocessed_data/df_final_raw_2015_2023.csv',index_col=0,parse_dates=['date'])

df = df.drop('Prog_in_MW',axis=1) # similar to target column, high danger of data leakage

cols_sd =[col for col in df.columns if 'SD' in col]
for col in cols_sd:
    # setting values < 0 (i.a. -999) to NaN
    df.loc[(df.loc[:,col] < 0),col] = pd.NA

cols_fd_fg = [col for col in df.columns if ('FD' in col)|('FG' in col)]
for col in cols_fd_fg:
    # setting values < 1 (-999 and 0) to NaN
    df.loc[(df.loc[:,col] < 1),col] = pd.NA

df.info()
display(df.head())

### Heatmap of data to look for correlation
Some of the weather data are highly correlating with respective columns over different stations, as it should be expected. Also, Bruttoleistung and Nettoleistung are likely the same. 

In [ ]:
fig,ax=plt.subplots(figsize=(15,15))
sns.heatmap(df.corr(),cmap="coolwarm",annot=True,ax=ax)
plt.xticks(rotation=-45)  # Rotate x-axis labels for readability

plt.tight_layout()  # Adjust spacing for readability
plt.show()

Example of one column with NaN values

In [ ]:
df_nan = df.loc[:,['date','05792_FG_STRAHL']].dropna()
df_nan.loc[:,'date']=df_nan.loc[:,'date'].map(pd.Timestamp.toordinal)
xn, yn = df_nan['date'],df_nan['05792_FG_STRAHL']
df_nan.loc[:,'date']=df_nan.loc[:,'date'].map(pd.Timestamp.fromordinal)

plt.scatter(xn, yn,s=6)

#calculate equation for trendline
zn = np.polyfit(xn, yn, 1)
pn = np.poly1d(zn)

#add trendline to plot
plt.plot(xn, pn(xn),color='red')

from scipy.stats import linregress
print('Slope of regression before imputation:',linregress(xn, yn).slope)

A short overview of respective row and column for a missing value, as well as the calculation for the filling value.

In [ ]:
cols_fg =[col for col in df.columns if 'FG' in col]
display(df.loc[(df['date'].dt.day==1) & (df['date'].dt.month==1),cols_fg])
print('Median of Column: ',df.loc[(df['date'].dt.day==1) & (df['date'].dt.month==1),'05792_FG_STRAHL'].median())
print('Median of Row: ',df.loc[(df.index==365) & (df['date'].dt.day==1) & (df['date'].dt.month==1),cols_fg].sum().median())
print('Mean value of both medians: ',((df.loc[(df['date'].dt.day==1) & (df['date'].dt.month==1),'05792_FG_STRAHL'].median())+(df.loc[(df.index==365) & (df['date'].dt.day==1) & (df['date'].dt.month==1),cols_fg].sum().median()))/2)

Imputation of missing values using the **mean** value of **median** of the row (actual day and respective physical parameter) and **median** of column (respective day over the years)

In [ ]:
# imputing implausible values
r, c = np.where(df.iloc[:,:-3].isna())
for i in range(len(r)):
    # getting all SD, FG or FD columns if applicable (identical physical measurements in Bavaria)
    cols_na = [col for col in df.columns if col.endswith(str(df.columns[c[i]])[5:])]

    # getting the median of specific day from corresponding columns
    med_row = df.loc[(df.index==r[i]) & (df['date'].dt.day==(df.iloc[r[i],0].day)) & (df['date'].dt.month==(df.iloc[r[i],0].month)),cols_na].sum().median()

    # getting the median of NaN column for specific day of the year
    med_col = df.loc[(df['date'].dt.day==(df.iloc[r[i],0].day)) & (df['date'].dt.month==(df.iloc[r[i],0].month)),df.columns[c[i]]].median()
    
    # calculating mean of both medians
    mean_v = np.nanmean(np.array([med_col,med_row]))

    # imputing
    df.iloc[r[i],c[i]] = mean_v

df.isna().sum() #no values were overseen

Example of one column (same as above) with imputed values

In [ ]:
df.loc[:,'date']=df.loc[:,'date'].map(pd.Timestamp.toordinal)
x, y = df['date'],df['05792_FG_STRAHL']
df.loc[:,'date']=df.loc[:,'date'].map(pd.Timestamp.fromordinal)
plt.scatter(x, y,s=6)

#calculate equation for trendline
z = np.polyfit(x, y, 1)
p = np.poly1d(z)

#add trendline to plot
plt.plot(x, p(x),color='red')

from scipy.stats import linregress
print('Slope of regression after imputation:',linregress(x, y).slope)

### New feature to be created: cumulative value of net- and gross production capacity

The cumulative value will show the actual capacity on each day. The base value for this calculation (values before 2015) were manually copied from the same source [Marktstammdatenregister](http://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/OeffentlicheEinheitenuebersicht?filter=Inbetriebnahmedatum%20der%20Einheit~lt~%2701.01.2015%27~and~Energietr%C3%A4ger~eq~%272495%27~and~Bundesland~eq~%271403%27)
Capacity values before 2015:
- 10.941.816 kWp gross
- 10.135.888 kWp net

Difference of the whole scripted calculation compared to automated calculation from source is less than 0.002 %, which is also explained by daily changing information on this platform.

In [ ]:
# creating new features for cumulative area of solarmodules for each day
df.loc[:,'Bruttoleistung_kumulativ'] = df.loc[:,'Bruttoleistung'].cumsum()+10941816
df.loc[:,'Nettoleistung_kumulativ'] = df.loc[:,'Nettoleistung'].cumsum()+10135888

# brutto vor 2024: 22.454.441 kW
# netto vor 2024: 20.404.484 kW
print('Difference in Brutto MW of extracted data and direct information from website:',22454441-round(df.iloc[-1,-2]),'(',round((22454441-round(df.iloc[-1,-2]))/22454441,3),'%)')
print('Difference in Netto MW of extracted data and direct information from website:',20404484-round(df.iloc[-1,-1]),'(',round((20404484-round(df.iloc[-1,-1]))/20404484,3),'%)')

In [ ]:
# saving the preprocessed data
df.to_csv('../Dataset/preprocessed_data/df_solar_energy_2015_2023.csv')

In [ ]:
# development of area of solarmodules from 2015-2023
df.plot(x='date',y='Bruttoleistung_kumulativ',figsize=(10,5)) 

### Insight on trends within the data

In [ ]:
summer = (df['date'].dt.month <= 8) & (df['date'].dt.month >= 5)
df.loc[:,'Bruttoleistung_kumulativ'] = df.loc[:,'Bruttoleistung_kumulativ']/1000
fig, ax1 = plt.subplots(figsize=(10,5))
df.loc[:,'date']=df.loc[:,'date'].map(pd.Timestamp.toordinal) #converting date from timestamp to ordinal
df_s = df.loc[summer,:]
df.loc[~summer,:].plot.scatter(x='date',y='Act_in_MW',ax=ax1,alpha=0.3,fontsize=10,color='grey',label='non-linear Regression')
ax1.grid(visible=None, which='major', axis='y')
sns.regplot(data=df_s, x='date', y='Act_in_MW',ax=ax1, color='blue',order=2, scatter_kws={'alpha': 0.2})
ax1.legend(loc=9,labelcolor='blue')
plt.ylabel("Actual Produced Energy in MW (daily)",fontsize=12) 
plt.xlabel("Date",fontsize=12) 
ax1.set_xlim(df.iloc[0,0], df.iloc[-1,0])
ax2 = ax1.twinx()
plt.title('Difference of Slopes in Trends for Capacity and Produced Energy',fontsize=15)
df.plot(x='date',y='Bruttoleistung_kumulativ',ax=ax2,color='black',label='Solar Panels Cumulative Sum',fontsize=10,lw=3) 
ax2.ticklabel_format(useOffset=False, style='plain')
xticks = ax1.get_xticks()
labels = [pd.Timestamp.fromordinal(int(label)).date() for label in xticks]
ax1.set_xticks(xticks)
ax1.xaxis.set_tick_params(labelrotation=-45)
ax1.set_xticklabels(labels,fontsize=10)
plt.ylabel("Cumulative Sum of Solar Panels in MWp",fontsize=12) 
df.loc[:,'date']=df.loc[:,'date'].map(pd.Timestamp.fromordinal) #converting date back to timestamp from ordinal
df.loc[:,'Bruttoleistung_kumulativ'] = df.loc[:,'Bruttoleistung_kumulativ']*1000
ax2.annotate(xy=(736750, 22700), text='_',fontsize=40,color='blue',xytext=(736900, 22700))
fig.tight_layout()
plt.show()

In [ ]:
# example overview of columns
df.loc[:,['date','05404_SD_STRAHL', '05404_FD_STRAHL', '05404_FG_STRAHL', 'Act_in_MW','Bruttoleistung','Bruttoleistung_kumulativ']]